Installation

In [82]:
pip install tensorflow==2.4.1

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [83]:
pip install tensorflow-gpu==2.4.1

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [84]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [85]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Import Dependencies

In [86]:
import tensorflow as tf 
import numpy as np
from matplotlib import pyplot as plt 
import cv2

Load Model

In [87]:
interpreter1 = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter1.allocate_tensors()

In [88]:
interpreter2 = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_thunder_3.tflite')
interpreter2.allocate_tensors()

In [89]:
interpreter1.get_input_details()

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1, 192, 192,   3]),
  'shape_signature': array([  1, 192, 192,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [90]:
interpreter1.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 312,
  'shape': array([ 1,  1, 17,  3]),
  'shape_signature': array([ 1,  1, 17,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

Draw Keypoints

In [91]:
# Here confidence_threshold is the threshhold above which we actually want to draw our keypoints
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    # In the above codeline, we are taking keypoints and multiplying them with the frame shape using np.multiply
    # And np.squeeze is for the easy access of the array

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0, 255, 0), -1)
            # cv2.circle(frame, x and y coordinates wrapping inside int, how big our circle would be, color of the circle, how thick the circle line(if we put it -1, it fills the circle with the given color))
            

Draw Edges

In [92]:
Edges = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [93]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape 
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]

        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
            # cv2.line(frame, first point, second point, color, line width)
    

Make Pose Detections

In [94]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis = 0), 192, 192)
    input_image = tf.cast(img, dtype = tf.float32)

    # Setup input and output
    input_details = interpreter1.get_input_details()
    output_details = interpreter1.get_output_details()

    # Make predictions
    interpreter1.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter1.invoke()
    keypoints_with_scores = interpreter1.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)

    # Rendering
    draw_connections(frame, keypoints_with_scores, Edges, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)

    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[[[[0.6935012  0.53564256 0.13455263]
   [0.6855171  0.53020626 0.13077396]
   [0.6749736  0.58309615 0.16775408]
   [0.6957958  0.50285906 0.09446079]
   [0.70128775 0.54340184 0.0897243 ]
   [0.73703134 0.4461348  0.0504249 ]
   [0.7376382  0.5875386  0.08956665]
   [0.70317405 0.50666094 0.05836937]
   [0.76927614 0.63293064 0.14050868]
   [0.6520025  0.5768043  0.06552875]
   [0.77206725 0.61577326 0.19291621]
   [0.7084478  0.4957701  0.04517007]
   [0.72700524 0.56036186 0.07598141]
   [0.799325   0.52302736 0.06427261]
   [0.850546   0.6510645  0.17803204]
   [0.82905906 0.5664791  0.07359245]
   [0.82649124 0.6455542  0.11974844]]]]
[[[[0.726913   0.5273627  0.21247107]
   [0.68591636 0.56882995 0.17947769]
   [0.6923596  0.48829874 0.22470936]
   [0.7069742  0.60669273 0.20598805]
   [0.71189374 0.4174016  0.14802766]
   [0.8224678  0.65829176 0.1943666 ]
   [0.7744502  0.3523153  0.24082473]
   [0.8690092  0.6638114  0.10833868]
   [0.8554419  0.4691914  0.09116367]
   [0.795

In [96]:
keypoints_with_scores[0][0]

"""
These shows 17 keypoints of our body, in the following order:
(1) Nose
(2) Left Eye
(3) Right Eye
(4) Left Ear
(5) Right Ear
(6) Left Shoulder
(7) Right Shoulder
(8) Left Elbow
(9) Right Elbow
(10) Left Wrist
(11) Right Wrist
(12) Left Hip
(13) Right Hip
(14) Left Knee
(15) Right Knee
(16) Left Ankle
(17) Right Ankle
"""

'\nThese shows 17 keypoints of our body, in the following order:\n(1) Nose\n(2) Left Eye\n(3) Right Eye\n(4) Left Ear\n(5) Right Ear\n(6) Left Shoulder\n(7) Right Shoulder\n(8) Left Elbow\n(9) Right Elbow\n(10) Left Wrist\n(11) Right Wrist\n(12) Left Hip\n(13) Right Hip\n(14) Left Knee\n(15) Right Knee\n(16) Left Ankle\n(17) Right Ankle\n'